In [1]:
# ============================================
#   暗闇迷路（盤面のみ）コンソール可視化
#   ◎ 全セル連結
#   ◎ 直線最大長 ≤ MAX_STRAIGHT
#   ◎ 2×2 通路禁止
#   ○ 分岐同士を再接続してループ適度に生成
# ============================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ---------- パラメータ ----------
SIZE          = 11     # 盤面の一辺長
MAX_STRAIGHT  = 3      # 通路が連続する最大セル数
LOOP_RATE     = 0.12   # 木構造にあとから追加するループの確率

# ---------- データ構造 ----------
@dataclass
class Maze:
    v: List[List[bool]]   # 垂直壁 True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]   # 水平壁 True=壁あり between (x,y)-(x,y+1)

# ---------- 基本ユーティリティ ----------
def neigh(x: int, y: int):
    if x > 0:         yield 'L', x-1, y
    if x < SIZE-1:    yield 'R', x+1, y
    if y > 0:         yield 'U', x, y-1
    if y < SIZE-1:    yield 'D', x, y+1

def remove_wall(m: Maze, x: int, y: int, d: str):
    if d == 'L': m.v[y][x-1] = False
    if d == 'R': m.v[y][x]   = False
    if d == 'U': m.h[y-1][x] = False
    if d == 'D': m.h[y][x]   = False

def wall(m: Maze, x: int, y: int, d: str) -> bool:
    if d == 'L': return m.v[y][x-1]
    if d == 'R': return m.v[y][x]
    if d == 'U': return m.h[y-1][x]
    if d == 'D': return m.h[y][x]
    return True

def causes_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    checks = []
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0,-1): checks.append((x+dx, y+dy))
    else:
        dy = -1 if d=='U' else 0
        for dx in (0,-1): checks.append((x+dx, y+dy))
    for cx, cy in checks:
        if 0 <= cx < SIZE-1 and 0 <= cy < SIZE-1:
            if (not m.v[cy][cx]   and not m.v[cy+1][cx] and
                not m.h[cy][cx]   and not m.h[cy][cx+1]):
                return True
    return False

# ---------- 1. 木構造 (直線≤MAX_STRAIGHT) ----------
def generate_tree() -> Maze:
    v = [[True]*(SIZE-1) for _ in range(SIZE)]
    h = [[True]*SIZE     for _ in range(SIZE-1)]
    visited = [[False]*SIZE for _ in range(SIZE)]
    sx, sy = random.randrange(SIZE), random.randrange(SIZE)
    stack: List[Tuple[int,int,str,int]] = [(sx, sy, '', 0)]
    visited[sy][sx] = True
    while stack:
        x, y, pd, run = stack[-1]
        nexts = [(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]
        # 直線長制限
        if run >= MAX_STRAIGHT:
            nexts = [t for t in nexts if t[0] != pd]
        # 2×2 通路禁止
        nexts = [t for t in nexts if not causes_open_square(Maze(v,h), x, y, t[0])]
        if nexts:
            d, nx, ny = random.choice(nexts)
            remove_wall(Maze(v,h), x, y, d)
            visited[ny][nx] = True
            stack.append((nx, ny, d, run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ---------- 2. ループ追加 ----------
def add_loops(m: Maze):
    edges = []
    for y in range(SIZE):
        for x in range(SIZE-1):
            if m.v[y][x]: edges.append((x,y,'R'))
    for y in range(SIZE-1):
        for x in range(SIZE):
            if m.h[y][x]: edges.append((x,y,'D'))
    random.shuffle(edges)

    deg = [[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1

    for x,y,d in edges:
        if random.random() > LOOP_RATE: continue
        nx, ny = (x+1,y) if d=='R' else (x,y+1)
        if deg[y][x] >= 3 or deg[ny][nx] >= 3: continue
        if causes_open_square(m, x, y, d):     continue
        remove_wall(m, x, y, d)
        deg[y][x]+=1; deg[ny][nx]+=1

# ---------- 3. 迷路生成 ----------
def generate_maze() -> Maze:
    while True:
        maze = generate_tree()
        add_loops(maze)
        if not causes_open_square(maze, 0,0,'R'):  # ざっくり再確認
            return maze

# ---------- 4. コンソール描画 ----------
def render(maze: Maze) -> str:
    lines = [' '+'_'*SIZE]
    for y in range(SIZE):
        row = ['|']
        for x in range(SIZE):
            ch = ' '
            if y==SIZE-1 or maze.h[y][x]: ch = '_' if ch==' ' else ch
            if x==SIZE-1 or maze.v[y][x]:
                row.append(ch+'|')
            else:
                row.append(ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ---------- 実行 ----------
if __name__ == "__main__":
    m = generate_maze()
    print(render(m))


 ___________
|   |_  |  _        | |
| |_  |_ _  |_ _| | | |
|   |  _  |    _ _| | |
| |_  |  _  |_  | |_  |
|_|   |  _ _ _  |_  | |
|  _|_ _| |   |    _| |
|_ _  |  _  |_ _ _ _ _|
|   |   |  _|  _  |_  |
| |_ _| |  _ _|  _|   |
| |   |_ _  |  _ _ _| |
|_ _|_ _ _ _|_|_ _ _ _|


In [3]:
# =====================================
#  2×2 通路絶対禁止版 ― 盤面可視化だけ
# =====================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ────────── 調整可能なパラメータ ──────────
SIZE          = 10     # 盤面の一辺長
MAX_STRAIGHT  = 3      # 通路の連続長上限
LOOP_RATE     = 0.12   # 木構造へ再接続する確率

# ────────── データ構造 ──────────
@dataclass
class Maze:
    v: List[List[bool]]   # 垂直壁 True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]   # 水平壁 True=壁あり between (x,y)-(x,y+1)

# ────────── 基本ユーティリティ ──────────
def neighbors(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def wall_exists(m: Maze, x: int, y: int, d: str) -> bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]   # 'D'

def carve(m: Maze, x: int, y: int, d: str):
    if d == 'L': m.v[y][x-1] = False
    if d == 'R': m.v[y][x]   = False
    if d == 'U': m.h[y-1][x] = False
    if d == 'D': m.h[y][x]   = False

# 2×2 通路が完成するかの判定
def creates_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    # carve 予定の壁の影響を受ける 2×2 ブロックの左上座標候補
    cands = []
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0, -1):
            cands.append((x+dx, y+dy))
    else:
        dy = -1 if d=='U' else 0
        for dx in (0, -1):
            cands.append((x+dx, y+dy))
    for cx, cy in cands:
        if 0 <= cx < SIZE-1 and 0 <= cy < SIZE-1:
            # 4 辺がすべて通路になるか確認
            v_left   = not (m.v[cy][cx]     if cx >=0         else True)
            v_right  = not (m.v[cy][cx+1]   if cx+1 < SIZE-1  else True)
            h_top    = not (m.h[cy][cx]     if cy >=0         else True)
            h_bottom = not (m.h[cy+1][cx]   if cy+1 < SIZE-1  else True)
            # carve 後をシミュレート
            if d=='L'  and cy in (y-1, y) and cx==x-1: v_left  = True
            if d=='R'  and cy in (y-1, y) and cx==x  : v_right = True
            if d=='U'  and cx in (x-1, x) and cy==y-1: h_top   = True
            if d=='D'  and cx in (x-1, x) and cy==y  : h_bottom= True
            if not v_left and not v_right and not h_top and not h_bottom:
                return True
    return False

# ────────── 1. 木構造（直線 ≤ MAX_STRAIGHT）──────────
def generate_tree() -> Maze:
    v = [[True]*(SIZE-1) for _ in range(SIZE)]
    h = [[True]*SIZE     for _ in range(SIZE-1)]
    visited = [[False]*SIZE for _ in range(SIZE)]

    sx, sy = random.randrange(SIZE), random.randrange(SIZE)
    stack: List[Tuple[int,int,str,int]] = [(sx, sy, '', 0)]
    visited[sy][sx] = True

    while stack:
        x, y, prev_d, run = stack[-1]
        nxt = [(d,nx,ny) for d,nx,ny in neighbors(x,y) if not visited[ny][nx]]

        if run >= MAX_STRAIGHT:
            nxt = [t for t in nxt if t[0] != prev_d]

        nxt = [t for t in nxt if not creates_open_square(Maze(v,h), x, y, t[0])]

        if nxt:
            d, nx, ny = random.choice(nxt)
            carve(Maze(v,h), x, y, d)
            visited[ny][nx] = True
            stack.append((nx, ny, d, run+1 if d==prev_d else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ────────── 2. ループ追加（2×2厳守・次数≤3）──────────
def add_loops(m: Maze):
    deg = [[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1

    cand = [(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]] + \
           [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(cand)

    for x,y,d in cand:
        if random.random() > LOOP_RATE: continue
        nx, ny = (x+1,y) if d=='R' else (x,y+1)

        if deg[y][x] >= 3 or deg[ny][nx] >= 3: continue
        if creates_open_square(m, x, y, d):    continue

        # 直線長を簡易チェック — carve 後に run が4を超えないかだけ検査
        if d == 'R':
            run = 1
            ix = x-1
            while ix>=0 and not m.v[y][ix]:   run+=1; ix-=1
            ix = x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run > MAX_STRAIGHT: continue
        else:
            run = 1
            iy = y-1
            while iy>=0 and not m.h[iy][x]:   run+=1; iy-=1
            iy = y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run > MAX_STRAIGHT: continue

        carve(m, x, y, d)
        deg[y][x]+=1; deg[ny][nx]+=1

# ────────── 3. 最終迷路生成 ──────────
def generate_maze() -> Maze:
    while True:
        m = generate_tree()
        add_loops(m)
        # 最終チェック: 2×2 通路が無いか
        if not any(
            not m.v[y][x] and not m.v[y+1][x] and
            not m.h[y][x] and not m.h[y][x+1]
            for y in range(SIZE-1) for x in range(SIZE-1)
        ):
            return m

# ────────── 4. コンソール描画 ──────────
def render(m: Maze) -> str:
    lines = [' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            if x==SIZE-1 or m.v[y][x]: row.append(ch+'|')
            else:                      row.append(ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ────────── 動作テスト ──────────
if __name__ == "__main__":
    random.seed()  # 必要に応じて固定
    maze = generate_maze()
    print(render(maze))


 __________
|  _  |     |   |  _|
|_  |  _| |_ _| |_  |
|  _| | |_  | |_  | |
|_ _ _  | | | |  _| |
|  _  |_ _| | |  _ _|
| | |_ _|  _|_ _    |
| |_ _   _|  _ _ _| |
|_ _  |_  |  _|  _  |
|  _|_  |_ _|   | | |
|_ _ _ _ _ _ _|_ _ _|


In [8]:
# ============================================
# 2×2 通路禁止 + 直線長 ≤ MAX_STRAIGHT
# 十字路 OK（次数上限なし）版
# ============================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# -------- 調整可能パラメータ --------
SIZE          = 10     # 盤面の一辺
MAX_STRAIGHT  = 3      # 通路が連続する最大セル数
LOOP_RATE     = 0.15   # 木構造に再接続する確率 (0～1)

# -------- 迷路データ構造 --------
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 (y,x) True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]  # 水平壁 (y,x) True=壁あり between (x,y)-(x,y+1)

# -------- 基本ユーティリティ --------
def neighbors(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def carve(m: Maze, x: int, y: int, d: str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

def wall(m: Maze, x: int, y: int, d: str)->bool:
    return m.v[y][x-1] if d=='L' else m.v[y][x] if d=='R' \
        else m.h[y-1][x] if d=='U' else m.h[y][x]          # 'D'

# ---- 2×2 通路ブロック出現判定 ----
def creates_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    cand=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        cand+=[(x+dx,y),(x+dx,y-1)]
    else:
        dy=-1 if d=='U' else 0
        cand+=[(x,y+dy),(x-1,y+dy)]
    for cx,cy in cand:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            if (not m.v[cy][cx] and not m.v[cy+1][cx] and
                not m.h[cy][cx] and not m.h[cy][cx+1]):
                return True
    return False

# ---- 1. 木構造生成 (直線≤MAX_STRAIGHT) ----
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0)]
    visited[sy][sx]=True
    while stack:
        x,y,pd,run=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neighbors(x,y) if not visited[ny][nx]]
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        nxt=[t for t in nxt if not creates_open_square(Maze(v,h),x,y,t[0])]
        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            visited[ny][nx]=True
            stack.append((nx,ny,d,run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ---- 2. ループ追加 (十字路許可) ----
def add_loops(m:Maze):
    edges=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(edges)
    for x,y,d in edges:
        if random.random()>LOOP_RATE: continue
        if creates_open_square(m,x,y,d):    continue
        # 直線長が破綻しないか簡易判定
        if d=='R':
            run=1
            ix=x-1
            while ix>=0 and not m.v[y][ix]: run+=1; ix-=1
            ix=x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run>MAX_STRAIGHT: continue
        else:
            run=1
            iy=y-1
            while iy>=0 and not m.h[iy][x]: run+=1; iy-=1
            iy=y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run>MAX_STRAIGHT: continue
        carve(m,x,y,d)

# ---- 3. 2×2 通路が一切無いか検証 ----
def has_open_square(m:Maze)->bool:
    return any(
        not m.v[y][x] and not m.v[y+1][x] and
        not m.h[y][x] and not m.h[y][x+1]
        for y in range(SIZE-1) for x in range(SIZE-1)
    )

# ---- 4. フル生成 ----
def generate_maze()->Maze:
    while True:
        m=generate_tree()
        add_loops(m)
        if not has_open_square(m):
            return m

# ---- 5. ASCII 描画 ----
def render(m:Maze)->str:
    lines=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)




In [12]:
# ---- 実行 ----
if __name__=="__main__":
    random.seed()          # 必要ならシード固定
    maze=generate_maze()
    print(render(maze))

 __________
|   |  _|   |    _  |
| |  _|  _|_ _|_  |_|
| |_|  _|  _ _  |_  |
|   | |   |   |_|   |
| | |_ _| | |_  | |_|
|_ _ _  | |_  | |_  |
|   |   |_  | |_ _ _|
| |_ _| |  _|_ _ _  |
|   |  _| | | |   | |
|_|_ _ _ _ _|_ _|_ _|


In [14]:
# =============================================================
# 2×2 通路禁止 & 直線通路 ≤ 4 マス 版（十字路 OK）
# =============================================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ───────── 調整用パラメータ ─────────
SIZE          = 10      # 盤面一辺
MAX_STRAIGHT  = 4       # 直線で続くセル数の上限
LOOP_RATE     = 0.15    # 木構造に後から追加するループ確率 (0-1)

# ───────── 迷路データ構造 ─────────
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 (y,x) True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]  # 水平壁 (y,x) True=壁あり between (x,y)-(x,y+1)

# ───────── ヘルパー ─────────
def neigh(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def wall(m: Maze, x:int, y:int, d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]                 # 'D'

def carve(m: Maze, x:int, y:int, d:str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

# 2×2 通路が出来るか判定
def makes_open_square(m: Maze, x:int, y:int, d:str) -> bool:
    # チェック対象の 2×2 ブロック左上候補
    cands=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): cands.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): cands.append((x+dx,y+dy))
    for cx,cy in cands:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0 = not m.v[cy][cx]
            v1 = not m.v[cy+1][cx]
            h0 = not m.h[cy][cx]
            h1 = not m.h[cy][cx+1]
            # carve 後を仮想反映
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

# ───────── 1. 木構造 (直線 ≤ MAX_STRAIGHT) ─────────
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0)]
    visited[sy][sx]=True
    while stack:
        x,y,pd,run=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        nxt=[t for t in nxt if not makes_open_square(Maze(v,h),x,y,t[0])]
        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            visited[ny][nx]=True
            stack.append((nx,ny,d, run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ───────── 2. ループ追加（十字路許可） ─────────
def add_loops(m: Maze):
    edges=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(edges)
    for x,y,d in edges:
        if random.random()>LOOP_RATE: continue
        # 直線長チェック
        if d=='R':
            run=1
            ix=x-1
            while ix>=0 and not m.v[y][ix]: run+=1; ix-=1
            ix=x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run>MAX_STRAIGHT: continue
        else:
            run=1
            iy=y-1
            while iy>=0 and not m.h[iy][x]: run+=1; iy-=1
            iy=y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run>MAX_STRAIGHT: continue
        if makes_open_square(m,x,y,d): continue
        carve(m,x,y,d)

# ───────── 3. 完全生成 & 最終 2×2 検査 ─────────
def generate_maze()->Maze:
    while True:
        maze=generate_tree()
        add_loops(maze)
        # 念のため最終スキャン
        if any(
            not maze.v[y][x] and not maze.v[y+1][x] and
            not maze.h[y][x] and not maze.h[y][x+1]
            for y in range(SIZE-1) for x in range(SIZE-1)
        ):
            continue
        return maze

# ───────── 4. コンソール描画 ─────────
def render(m:Maze)->str:
    lines=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ───────── 実行例 ─────────
if __name__=="__main__":
    random.seed()  # 必要に応じて固定
    maze=generate_maze()
    print(render(maze))


 __________
|  _|  _ _    |_    |
| |  _|   | |  _ _| |
|_ _  | | | |_ _ _  |
|    _|   |_ _ _  | |
| |_  | |_ _ _  | |_|
| |_ _  | |  _|_  | |
| |   | | |  _  | | |
|  _| | |   | |  _  |
|_    |_| |  _    | |
|_ _|_ _ _|_ _ _|_ _|


In [15]:
# ---------------- 追加: 直線長を正確に評価 -----------------
def max_run_after_carve(m: Maze, x: int, y: int, d: str) -> int:
    """
    壁 (x,y,d) を壊した場合に出来る
    “横方向と縦方向で最長になる直線通路長” を返す。
    どちらか一方でも MAX_STRAIGHT を超えたら NG 判定に使う。
    """
    # 仮想的に壊した後の run を数える関数
    def run_len_horz(px: int, py: int):
        run = 1
        ix = px-1
        while ix >= 0 and not m.v[py][ix]:
            run += 1; ix -= 1
        ix = px
        while ix < SIZE-1 and not m.v[py][ix]:
            run += 1; ix += 1
        return run

    def run_len_vert(px: int, py: int):
        run = 1
        iy = py-1
        while iy >= 0 and not m.h[iy][px]:
            run += 1; iy -= 1
        iy = py
        while iy < SIZE-1 and not m.h[iy][px]:
            run += 1; iy += 1
        return run

    # carve 後に通路になる 2 セルの座標
    if d == 'R':
        cells = [(x, y), (x+1, y)]
    elif d == 'L':
        cells = [(x-1, y), (x, y)]
    elif d == 'D':
        cells = [(x, y), (x, y+1)]
    else:  # 'U'
        cells = [(x, y-1), (x, y)]

    max_run = 0
    for cx, cy in cells:
        max_run = max(max_run,
                      run_len_horz(cx, cy),
                      run_len_vert(cx, cy))
    return max_run


In [20]:
# ==========================================================
#  十字路 OK / 直線通路 ≤ 4 マス / 2×2 通路禁止 迷路ジェネレータ
# ==========================================================
import random
from dataclasses import dataclass
from typing import List, Tuple

# ───────── 調整可能パラメータ ─────────
SIZE          = 10      # 盤面の一辺
MAX_STRAIGHT  = 5       # 通路の最大直線長
LOOP_RATE     = 0.25    # 木構造に再接続する確率 (0–1)

# ───────── 迷路データ構造 ─────────
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]  # 水平壁 True=壁あり between (x,y)-(x,y+1)

# ───────── 基本ヘルパー ─────────
def neigh(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def wall(m: Maze, x: int, y: int, d: str) -> bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]                 # 'D'

def carve(m: Maze, x: int, y: int, d: str):
    if d == 'L': m.v[y][x-1] = False
    if d == 'R': m.v[y][x]   = False
    if d == 'U': m.h[y-1][x] = False
    if d == 'D': m.h[y][x]   = False

# ───────── 2×2 通路判定 ─────────
def makes_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    """壁 (x,y,d) を壊すと 2×2 通路が出来るか"""
    cands=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): cands.append((x+dx, y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): cands.append((x+dx, y+dy))
    for cx,cy in cands:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0 = not m.v[cy][cx]
            v1 = not m.v[cy+1][cx]
            h0 = not m.h[cy][cx]
            h1 = not m.h[cy][cx+1]
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

# ───────── 直線長シミュレーション ─────────
def max_run_after_carve(m: Maze, x: int, y: int, d: str) -> int:
    """壁を壊した後の縦横いずれかの最大直線通路長を返す"""
    def run_h(px:int,py:int):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px:int,py:int):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run

    # carve 後 通路セルとなる 2 点
    cells = [(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else: cells.append((x,y-1))

    max_run=0
    for cx,cy in cells:
        max_run=max(max_run, run_h(cx,cy), run_v(cx,cy))
    return max_run

# ───────── 1. 木構造生成（直線≤MAX_STRAIGHT）─────────
def generate_tree() -> Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0)]
    visited[sy][sx]=True
    while stack:
        x,y,pd,run=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        nxt=[t for t in nxt if not makes_open_square(Maze(v,h),x,y,t[0])]
        nxt=[t for t in nxt if max_run_after_carve(Maze(v,h),x,y,t[0])<=MAX_STRAIGHT]
        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            visited[ny][nx]=True
            stack.append((nx,ny,d, run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ───────── 2. ループ追加（直線長保持）─────────
def add_loops(m: Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d):                      continue
        if max_run_after_carve(m,x,y,d) > MAX_STRAIGHT:    continue
        carve(m,x,y,d)

# ───────── 3. 最終直線/2×2 検査 ─────────
def violates_constraints(m: Maze) -> bool:
    # 直線長
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run=run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run=run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 2×2 通路
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    return False

# ───────── 4. 迷路生成 ─────────
def generate_maze() -> Maze:
    while True:
        maze=generate_tree()
        add_loops(maze)
        if not violates_constraints(maze):
            return maze

# ───────── 5. コンソール描画 ─────────
def render(m:Maze)->str:
    lines=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ───────── 実行例 ─────────
if __name__=="__main__":
    random.seed()  # 乱数シードを固定したい場合は数値を入れる
    maze=generate_maze()
    print(render(maze))


 __________
|  _ _ _|  _ _  |   |
|   |  _ _|   |  _| |
| |_ _|_    |  _   _|
| |  _ _  | |_|  _  |
|_ _| |  _ _  |   | |
| |   |_   _|  _|  _|
|  _|_    |  _  |   |
| |  _  |  _|_   _| |
| | |   | |  _ _ _  |
|_ _ _|_|_ _|_ _ _ _|


In [39]:
# ───────── 調整可能パラメータ ─────────
SIZE          = 5      # 盤面の一辺
MAX_STRAIGHT  = 3       # 通路の最大直線長
LOOP_RATE     = 0.25    # 木構造に再接続する確率 (0–1)

# ───────── 実行例 ─────────
if __name__=="__main__":
    random.seed()  # 乱数シードを固定したい場合は数値を入れる
    maze=generate_maze()
    print(render(maze))

 _____
|  _  |   |
| |_ _ _| |
|_  |   |_|
|  _  | | |
|_ _ _|_ _|


In [43]:
"""
暗闇迷路ジェネレータ  (十字路許可 / 2×2通路禁止)
────────────────────────────────────────
制約
  ✅ 直線通路は MAX_STRAIGHT 以下
  ✅ 2×2 の完全開放通路はゼロ
  ✅ 分岐 (degree≠2) 間の距離は MAX_BRANCH_DIST 以下
     └「degree==2 のセル」が 連続して 7 個以上続かない
  ✅ 盤面は連結。木構造を生成後、確率 LOOP_RATE で再接続
出力
  ./maze_dungeon/maze_<SIZE>_<timestamp>.json
  JSON は {size, v_walls, h_walls} だけ（Start/Goal は後工程）
"""

# ───────── 調整パラメータ ─────────
SIZE             = 10      # 盤面一辺
MAX_STRAIGHT     = 4       # 通路直線長上限
MAX_BRANCH_DIST  = 6       # 分岐間の最大距離 (セル数)
LOOP_RATE        = 0.25    # ループ追加確率

# ────────────────────────────────
import random, os, json, datetime
from dataclasses import dataclass
from typing import List, Tuple

Cell = Tuple[int, int]

@dataclass
class Maze:
    v: List[List[bool]]    # 垂直壁 (y,x) True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]    # 水平壁 (y,x) True=壁あり between (x,y)-(x,y+1)

# ───────── ヘルパー ─────────
def neigh(x:int,y:int):
    if x>0:        yield 'L',x-1,y
    if x<SIZE-1:   yield 'R',x+1,y
    if y>0:        yield 'U',x,y-1
    if y<SIZE-1:   yield 'D',x,y+1

def carve(m:Maze,x:int,y:int,d:str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

def wall(m:Maze,x:int,y:int,d:str)->bool:
    return m.v[y][x-1] if d=='L' else\
           m.v[y][x]   if d=='R' else\
           m.h[y-1][x] if d=='U' else\
           m.h[y][x]

def makes_open_square(m:Maze,x:int,y:int,d:str)->bool:
    """壁を壊して 2×2 通路が出来るか"""
    checks=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): checks.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): checks.append((x+dx,y+dy))
    for cx,cy in checks:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx];      v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx];      h1=not m.h[cy][cx+1]
            # carve 後を仮反映
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

def max_run_after_carve(m:Maze,x:int,y:int,d:str)->int:
    """壊したあと縦横方向で最大何マス連続するか"""
    def run_h(px:int,py:int):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px:int,py:int):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else: cells.append((x,y-1))
    return max(max(run_h(cx,cy),run_v(cx,cy)) for cx,cy in cells)

# ───────── 1) 木構造生成 ─────────
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0,0)]  # (x,y,prev_dir,run_len,dist_from_branch)
    visited[sy][sx]=True
    degree=[[0]*SIZE for _ in range(SIZE)]

    while stack:
        x,y,pd,run,dist=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]

        # 直線長制限
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        # 分岐最大距離: dist が MAX_BRANCH_DIST なら “必ず分岐” させる
        if dist>=MAX_BRANCH_DIST and pd:
            # 必ず枝分かれ → 現セルで degree を増やす必要がある
            nxt=[t for t in nxt if len(nxt)>=2]  # 2 方向以上残っていれば OK

        # 2×2 / 直線事前チェック
        nxt=[t for t in nxt
             if not makes_open_square(Maze(v,h),x,y,t[0])
             and max_run_after_carve(Maze(v,h),x,y,t[0])<=MAX_STRAIGHT]

        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            degree[y][x]+=1; degree[ny][nx]+=1
            new_dist = 0 if degree[ny][nx]>=3 else dist+1
            new_run  = run+1 if d==pd else 1
            stack.append((nx,ny,d,new_run,new_dist))
            visited[ny][nx]=True
        else:
            stack.pop()
    return Maze(v,h)

# ───────── 2) ループ追加 ─────────
def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d): continue
        if max_run_after_carve(m,x,y,d)>MAX_STRAIGHT: continue
        carve(m,x,y,d)

# ───────── 3) 検証 ─────────
def violates_constraints(m:Maze)->bool:
    # 直線長
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run=run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run=run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 2×2 通路
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    # 分岐最大距離
    deg=[[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1
    # 深さ優先で degree==2 の連鎖を数える
    visited=set()
    for y in range(SIZE):
        for x in range(SIZE):
            if (x,y) in visited: continue
            if deg[y][x]==2:
                # traverse chain
                chain=[(x,y)]
                visited.add((x,y))
                # 双方向探索
                ends=[(x,y)]
                while ends:
                    cx,cy=ends.pop()
                    for d,nx,ny in neigh(cx,cy):
                        if wall(m,cx,cy,d): continue
                        if (nx,ny) in visited: continue
                        if deg[ny][nx]==2:
                            chain.append((nx,ny))
                            visited.add((nx,ny))
                            ends.append((nx,ny))
                if len(chain)>MAX_BRANCH_DIST:
                    return True
            else:
                visited.add((x,y))
    return False

# ───────── 4) 迷路生成 ─────────
def generate_maze()->Maze:
    while True:
        maze=generate_tree()
        add_loops(maze)
        if not violates_constraints(maze):
            return maze

# ───────── 5) JSON 出力 ─────────
def maze_to_json(m:Maze)->dict:
    v=[[x,y] for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]
    h=[[x,y] for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    return {"size":SIZE,"v_walls":v,"h_walls":h}

def save_maze_json(m:Maze):
    os.makedirs("maze_dungeon",exist_ok=True)
    ts=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    fname=f"maze_{SIZE}_{ts}.json"
    with open(os.path.join("maze_dungeon",fname),"w",encoding="utf-8") as f:
        json.dump(maze_to_json(m),f,ensure_ascii=False,indent=2)
    print(f"✅ saved: maze_dungeon/{fname}")

# ───────── 6) ASCII 可視化 ─────────
def render(m:Maze)->str:
    out=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        out.append(''.join(row))
    return '\n'.join(out)

# ───────── 7) 実行サンプル ─────────
if __name__=="__main__":
    random.seed()          # 任意で固定
    maze=generate_maze()
    print(render(maze))
    # save_maze_json(maze)


 __________
|_|_|  _|  _ _ _|_| |
|  _|  _    |_     _|
|_ _ _  |_|_ _| | |_|
| | |  _|    _  |_  |
| | |    _|  _ _|_  |
|_ _  |_ _ _|_| |_|_|
|_|  _ _|_|_| |_|_|_|
|_|_  |_|_ _|_|_| |_|
|  _  |_|  _|_| |_ _|
|_ _ _|_|_|_|_ _|_|_|


In [150]:
"""
暗闇迷路ジェネレータ  (孤立セル禁止版)
────────────────────────────────────
制約
  ✓ 直線通路長 ≤ 4
  ✓ 2×2 通路ブロック無し
  ✓ 分岐間距離 ≤ 6 マス
  ✓ 十字路許可
  ✓ 盤面は完全連結（孤立セル無し） ★New
"""

# ─── 調整パラメータ ───
SIZE             = 6
MAX_STRAIGHT     = 4
MAX_BRANCH_DIST  = 7
LOOP_RATE        = 0.25

# ─── ライブラリ ───
import random, os, json, datetime
from dataclasses import dataclass
from typing import List, Tuple, Set

# ─── データ構造 ───
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 (y,x)
    h: List[List[bool]]  # 水平壁 (y,x)

Cell = Tuple[int, int]

# ─── ヘルパー ───
def neigh(x:int,y:int):
    if x>0:        yield 'L',x-1,y
    if x<SIZE-1:   yield 'R',x+1,y
    if y>0:        yield 'U',x,y-1
    if y<SIZE-1:   yield 'D',x,y+1

def carve(m:Maze,x:int,y:int,d:str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

def wall(m:Maze,x:int,y:int,d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]                 # 'D'

def makes_open_square(m:Maze,x:int,y:int,d:str)->bool:
    c=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): c.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): c.append((x+dx,y+dy))
    for cx,cy in c:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx]; v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx]; h1=not m.h[cy][cx+1]
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

def max_run_after_carve(m:Maze,x:int,y:int,d:str)->int:
    def run_h(px,py):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px,py):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else: cells.append((x,y-1))
    return max(max(run_h(cx,cy),run_v(cx,cy)) for cx,cy in cells)

# ─── 1) 木構造生成 ───
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    degree=[[0]*SIZE for _ in range(SIZE)]

    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0,0)]                # (x,y,prev_dir,run,dist_from_branch)
    visited[sy][sx]=True
    while stack:
        x,y,pd,run,dist=stack[-1]
        cand=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]

        if run>=MAX_STRAIGHT:
            cand=[t for t in cand if t[0]!=pd]
        if dist>=MAX_BRANCH_DIST and pd:
            # 必ず分岐（degreeが3以上になる）候補のみ保持
            cand=[t for t in cand if len(cand)>=2]

        cand=[t for t in cand
              if not makes_open_square(Maze(v,h),x,y,t[0])
              and max_run_after_carve(Maze(v,h),x,y,t[0])<=MAX_STRAIGHT]

        if cand:
            d,nx,ny=random.choice(cand)
            carve(Maze(v,h),x,y,d)
            degree[y][x]+=1; degree[ny][nx]+=1
            visited[ny][nx]=True
            new_dist=0 if degree[ny][nx]>=3 else dist+1
            new_run = run+1 if d==pd else 1
            stack.append((nx,ny,d,new_run,new_dist))
        else:
            stack.pop()
    return Maze(v,h)

# ─── 2) ループ追加 ───
def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d): continue
        if max_run_after_carve(m,x,y,d)>MAX_STRAIGHT: continue
        carve(m,x,y,d)

# ─── 3) 連結性チェック ───
def is_connected(m:Maze)->bool:
    q=[(0,0)]
    seen:set[Cell]=set(q)
    while q:
        x,y=q.pop()
        for d,nx,ny in neigh(x,y):
            if (nx,ny) in seen: continue
            if wall(m,x,y,d):    continue
            seen.add((nx,ny)); q.append((nx,ny))
    return len(seen)==SIZE*SIZE

# ─── 4) その他の制約検証 ───
def violates_constraints(m:Maze)->bool:
    # 直線
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run=run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run=run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 2×2
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    # 分岐距離
    deg=[[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1
    visited=set()
    for y in range(SIZE):
        for x in range(SIZE):
            if (x,y) in visited: continue
            if deg[y][x]==2:
                chain=[(x,y)]; visited.add((x,y))
                ends=[(x,y)]
                while ends:
                    cx,cy=ends.pop()
                    for d,nx,ny in neigh(cx,cy):
                        if wall(m,cx,cy,d): continue
                        if (nx,ny) in visited: continue
                        if deg[ny][nx]==2:
                            chain.append((nx,ny)); visited.add((nx,ny)); ends.append((nx,ny))
                if len(chain)>MAX_BRANCH_DIST: return True
            else:
                visited.add((x,y))
    return False

# ─── 5) 迷路生成 ───
def generate_maze()->Maze:
    while True:
        maze=generate_tree()
        add_loops(maze)
        if is_connected(maze) and not violates_constraints(maze):
            return maze


# ─── 7) ASCII 可視化 ───
def render(m:Maze)->str:
    out=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        out.append(''.join(row))
    return '\n'.join(out)

# ─── 8) 実行サンプル ───
if __name__=="__main__":
    random.seed()
    maze=generate_maze()
    print(render(maze))


 ______
| |  _ _  | |
|    _| |  _|
|_|  _   _| |
|_ _  |_    |
| |       | |
|_ _|_|_|_ _|


In [163]:
# ─── 調整パラメータ ───
SIZE             = 10
MAX_STRAIGHT     = 10
MAX_BRANCH_DIST  = 8
LOOP_RATE        = 0.4

# ─── 8) 実行サンプル ───
if __name__=="__main__":
    random.seed()
    maze=generate_maze()
    print(render(maze))

 __________
|_   _    |  _|_   _|
|  _  | |    _  |  _|
|_|_ _  | |_ _ _    |
| |   |_|_| |   |_| |
| | |_|  _|  _|_|_  |
| | |  _   _ _  |   |
|_   _|_   _  | | | |
|     |  _ _  |  _| |
| | |     | |_|  _ _|
|_ _ _|_|_ _ _ _ _ _|


In [154]:
"""
Batch Maze Generator ― progress display
────────────────────────────────────────
制約
  • 全セル連結（孤立セルなし）
  • 直線通路 ≤ MAX_STRAIGHT
  • 2×2 通路禁止
  • 分岐間距離 ≤ MAX_BRANCH_DIST
  • 十字路許可・ループ率 LOOP_RATE
出力
  ./maze_dungeon/maze_<SIZE>.json
  各迷路 JSON には下記パラメータも同梱:
    max_straight, max_branch_dist, loop_rate
"""

# ─── adjustable parameters ───
SIZE             = 5       # board width = height
MAX_STRAIGHT     = 3       # longest straight corridor
MAX_BRANCH_DIST  = 4       # max cells between branch nodes
LOOP_RATE        = 0.25    # probability to add a loop edge
MAZE_COUNT       = 25      # ← change here to generate more / fewer mazes
SEED             = None    # e.g. 123 for reproducible output

# ─────────────────────────────
import random, os, json, datetime
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set

Cell = Tuple[int, int]

@dataclass
class Maze:
    v: List[List[bool]]     # vert walls (y,x) True = wall
    h: List[List[bool]]     # horz walls (y,x) True = wall

# ─── helper functions ───
def neigh(x:int, y:int):
    if x>0:        yield 'L', x-1, y
    if x<SIZE-1:   yield 'R', x+1, y
    if y>0:        yield 'U', x, y-1
    if y<SIZE-1:   yield 'D', x, y+1

def carve(m:Maze, x:int, y:int, d:str):
    if d=='L': m.v[y][x-1] = False
    elif d=='R': m.v[y][x] = False
    elif d=='U': m.h[y-1][x] = False
    else: m.h[y][x] = False            # 'D'

def wall(m:Maze, x:int, y:int, d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]

# 2×2 open-square prevention
def makes_open_square(m:Maze, x:int, y:int, d:str)->bool:
    blocks=[]
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0,-1): blocks.append((x+dx, y+dy))
    else:                             # U/D
        dy = -1 if d=='U' else 0
        for dx in (0,-1): blocks.append((x+dx, y+dy))

    for cx,cy in blocks:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx];      v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx];      h1=not m.h[cy][cx+1]
            # simulate carve
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

# longest straight run after hypothetical carve
def max_run_after_carve(m:Maze,x:int,y:int,d:str)->int:
    def run_h(px,py):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px,py):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else: cells.append((x,y-1))
    return max(max(run_h(cx,cy), run_v(cx,cy)) for cx,cy in cells)

# ─── core generation ───
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    degree=[[0]*SIZE for _ in range(SIZE)]

    sx,sy=random.randrange(SIZE), random.randrange(SIZE)
    stack=[(sx,sy,'',0,0)]    # (x,y,prev_dir,run_len,dist_from_branch)
    visited[sy][sx]=True

    while stack:
        x,y,pd,run,dist = stack[-1]
        cand=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]

        if run>=MAX_STRAIGHT:
            cand=[t for t in cand if t[0]!=pd]
        if dist>=MAX_BRANCH_DIST and pd:
            cand=[t for t in cand if len(cand)>=2]

        cand=[t for t in cand
              if not makes_open_square(Maze(v,h),x,y,t[0])
              and max_run_after_carve(Maze(v,h),x,y,t[0])<=MAX_STRAIGHT]

        if cand:
            d,nx,ny = random.choice(cand)
            carve(Maze(v,h),x,y,d)
            degree[y][x]+=1; degree[ny][nx]+=1
            visited[ny][nx]=True
            nxt_dist = 0 if degree[ny][nx]>=3 else dist+1
            nxt_run  = run+1 if d==pd else 1
            stack.append((nx,ny,d,nxt_run,nxt_dist))
        else:
            stack.pop()
    return Maze(v,h)

def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d):  continue
        if max_run_after_carve(m,x,y,d)>MAX_STRAIGHT: continue
        carve(m,x,y,d)

def is_connected(m:Maze)->bool:
    q=[(0,0)]; seen=set(q)
    while q:
        x,y=q.pop()
        for d,nx,ny in neigh(x,y):
            if (nx,ny) in seen: continue
            if wall(m,x,y,d):    continue
            seen.add((nx,ny)); q.append((nx,ny))
    return len(seen)==SIZE*SIZE

def violates(m:Maze)->bool:
    # straight-run check
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run = run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run = run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 2×2
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    # branch-distance check (degree==2 chain)
    deg=[[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1
    visited:set[Cell]=set()
    for y in range(SIZE):
        for x in range(SIZE):
            if (x,y) in visited: continue
            if deg[y][x]==2:
                chain=[(x,y)]; visited.add((x,y))
                ends=[(x,y)]
                while ends:
                    cx,cy=ends.pop()
                    for d,nx,ny in neigh(cx,cy):
                        if (nx,ny) in visited or wall(m,cx,cy,d): continue
                        if deg[ny][nx]==2:
                            chain.append((nx,ny)); visited.add((nx,ny)); ends.append((nx,ny))
                if len(chain)>MAX_BRANCH_DIST:
                    return True
            else:
                visited.add((x,y))
    return False

def generate_maze()->Maze:
    while True:
        m=generate_tree()
        add_loops(m)
        if is_connected(m) and not violates(m):
            return m

# ─── conversion & save ───
def maze_to_json(m:Maze, idx:int)->Dict:
    v=[[x,y] for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]
    h=[[x,y] for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    return {
        "id": f"maze{idx:03}",
        "size": SIZE,
        "max_straight": MAX_STRAIGHT,
        "max_branch_dist": MAX_BRANCH_DIST,
        "loop_rate": LOOP_RATE,
        "v_walls": v,
        "h_walls": h
    }

def generate_batch() -> List[Dict]:
    if SEED is not None:
        random.seed(SEED)
    mazes=[]
    for i in range(1, MAZE_COUNT+1):
        print(f"[{i}/{MAZE_COUNT}] generating maze...", end="", flush=True)
        m=generate_maze()
        mazes.append(maze_to_json(m,i))
        print(" done")
    return mazes

def save_batch(mazes:List[Dict]):
    os.makedirs("maze_dungeon",exist_ok=True)
    file_path=os.path.join("maze_dungeon", f"maze_{SIZE}.json")
    with open(file_path,"w",encoding="utf-8") as f:
        json.dump(mazes,f,ensure_ascii=False,indent=2)
    print(f"\n✅ Saved {len(mazes)} mazes to {file_path}")

# ─── run ───
if __name__=="__main__":
    batch=generate_batch()
    save_batch(batch)


[1/25] generating maze... done
[2/25] generating maze... done
[3/25] generating maze... done
[4/25] generating maze... done
[5/25] generating maze... done
[6/25] generating maze... done
[7/25] generating maze... done
[8/25] generating maze... done
[9/25] generating maze... done
[10/25] generating maze... done
[11/25] generating maze... done
[12/25] generating maze... done
[13/25] generating maze... done
[14/25] generating maze... done
[15/25] generating maze... done
[16/25] generating maze... done
[17/25] generating maze... done
[18/25] generating maze... done
[19/25] generating maze... done
[20/25] generating maze... done
[21/25] generating maze... done
[22/25] generating maze... done
[23/25] generating maze... done
[24/25] generating maze... done
[25/25] generating maze... done

✅ Saved 25 mazes to maze_dungeon\maze_5.json


In [3]:
"""
Batch Maze Generator ― progress display
────────────────────────────────────────
制約
  • 全セル連結（孤立セルなし）
  • 直線通路 ≤ MAX_STRAIGHT
  • 2×2 通路禁止
  • 分岐間距離 ≤ MAX_BRANCH_DIST
  • 十字路許可・ループ率 LOOP_RATE
出力
  ./maze_dungeon/maze_<SIZE>.json
  各迷路 JSON には下記パラメータも同梱:
    max_straight, max_branch_dist, loop_rate
"""

# ─── adjustable parameters ───
SIZE             = 9       # board width = height
MAX_STRAIGHT     = 9       # longest straight corridor
MAX_BRANCH_DIST  = 9        # max cells between branch nodes
LOOP_RATE        = 0.2      # probability to add a loop edge
MAZE_COUNT       = 1       # ← change here to generate more / fewer mazes
SEED             = 9     # e.g. 123 for reproducible output

# ─────────────────────────────
import random, os, json, datetime
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set

Cell = Tuple[int, int]

@dataclass
class Maze:
    v: List[List[bool]]     # vert walls (y,x) True = wall
    h: List[List[bool]]     # horz walls (y,x) True = wall

# ─── helper functions ───
def neigh(x:int, y:int):
    if x>0:        yield 'L', x-1, y
    if x<SIZE-1:   yield 'R', x+1, y
    if y>0:        yield 'U', x, y-1
    if y<SIZE-1:   yield 'D', x, y+1

def carve(m:Maze, x:int, y:int, d:str):
    if d=='L': m.v[y][x-1] = False
    elif d=='R': m.v[y][x] = False
    elif d=='U': m.h[y-1][x] = False
    else: m.h[y][x] = False            # 'D'

def wall(m:Maze, x:int, y:int, d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]

# 2×2 open-square prevention
def makes_open_square(m:Maze, x:int, y:int, d:str)->bool:
    blocks=[]
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0,-1): blocks.append((x+dx, y+dy))
    else:                             # U/D
        dy = -1 if d=='U' else 0
        for dx in (0,-1): blocks.append((x+dx, y+dy))

    for cx,cy in blocks:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0=not m.v[cy][cx];      v1=not m.v[cy+1][cx]
            h0=not m.h[cy][cx];      h1=not m.h[cy][cx+1]
            # simulate carve
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

# longest straight run after hypothetical carve
def max_run_after_carve(m:Maze,x:int,y:int,d:str)->int:
    def run_h(px,py):
        run=1; ix=px-1
        while ix>=0 and not m.v[py][ix]: run+=1; ix-=1
        ix=px
        while ix<SIZE-1 and not m.v[py][ix]: run+=1; ix+=1
        return run
    def run_v(px,py):
        run=1; iy=py-1
        while iy>=0 and not m.h[iy][px]: run+=1; iy-=1
        iy=py
        while iy<SIZE-1 and not m.h[iy][px]: run+=1; iy+=1
        return run
    cells=[(x,y)]
    if d=='R': cells.append((x+1,y))
    elif d=='L': cells.append((x-1,y))
    elif d=='D': cells.append((x,y+1))
    else: cells.append((x,y-1))
    return max(max(run_h(cx,cy), run_v(cx,cy)) for cx,cy in cells)

# ─── core generation ───
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    degree=[[0]*SIZE for _ in range(SIZE)]

    sx,sy=random.randrange(SIZE), random.randrange(SIZE)
    stack=[(sx,sy,'',0,0)]    # (x,y,prev_dir,run_len,dist_from_branch)
    visited[sy][sx]=True

    while stack:
        x,y,pd,run,dist = stack[-1]
        cand=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]

        if run>=MAX_STRAIGHT:
            cand=[t for t in cand if t[0]!=pd]
        if dist>=MAX_BRANCH_DIST and pd:
            cand=[t for t in cand if len(cand)>=2]

        cand=[t for t in cand
              if not makes_open_square(Maze(v,h),x,y,t[0])
              and max_run_after_carve(Maze(v,h),x,y,t[0])<=MAX_STRAIGHT]

        if cand:
            d,nx,ny = random.choice(cand)
            carve(Maze(v,h),x,y,d)
            degree[y][x]+=1; degree[ny][nx]+=1
            visited[ny][nx]=True
            nxt_dist = 0 if degree[ny][nx]>=3 else dist+1
            nxt_run  = run+1 if d==pd else 1
            stack.append((nx,ny,d,nxt_run,nxt_dist))
        else:
            stack.pop()
    return Maze(v,h)

def add_loops(m:Maze):
    walls=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)
    for x,y,d in walls:
        if random.random()>LOOP_RATE: continue
        if makes_open_square(m,x,y,d):  continue
        if max_run_after_carve(m,x,y,d)>MAX_STRAIGHT: continue
        carve(m,x,y,d)

def is_connected(m:Maze)->bool:
    q=[(0,0)]; seen=set(q)
    while q:
        x,y=q.pop()
        for d,nx,ny in neigh(x,y):
            if (nx,ny) in seen: continue
            if wall(m,x,y,d):    continue
            seen.add((nx,ny)); q.append((nx,ny))
    return len(seen)==SIZE*SIZE

def violates(m:Maze)->bool:
    # straight-run check
    for y in range(SIZE):
        run=1
        for x in range(SIZE-1):
            run = run+1 if not m.v[y][x] else 1
            if run>MAX_STRAIGHT: return True
    for x in range(SIZE):
        run=1
        for y in range(SIZE-1):
            run = run+1 if not m.h[y][x] else 1
            if run>MAX_STRAIGHT: return True
    # 2×2
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    # branch-distance check (degree==2 chain)
    deg=[[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1
    visited:set[Cell]=set()
    for y in range(SIZE):
        for x in range(SIZE):
            if (x,y) in visited: continue
            if deg[y][x]==2:
                chain=[(x,y)]; visited.add((x,y))
                ends=[(x,y)]
                while ends:
                    cx,cy=ends.pop()
                    for d,nx,ny in neigh(cx,cy):
                        if (nx,ny) in visited or wall(m,cx,cy,d): continue
                        if deg[ny][nx]==2:
                            chain.append((nx,ny)); visited.add((nx,ny)); ends.append((nx,ny))
                if len(chain)>MAX_BRANCH_DIST:
                    return True
            else:
                visited.add((x,y))
    return False

def generate_maze()->Maze:
    while True:
        m=generate_tree()
        add_loops(m)
        if is_connected(m) and not violates(m):
            return m

# ─── conversion & save ───
def maze_to_json(m:Maze, idx:int)->Dict:
    v=[[x,y] for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]
    h=[[x,y] for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    return {
        "id": f"maze{idx:03}",
        "size": SIZE,
        "max_straight": MAX_STRAIGHT,
        "max_branch_dist": MAX_BRANCH_DIST,
        "loop_rate": LOOP_RATE,
        "v_walls": v,
        "h_walls": h
    }

# ─── helper functions ───
def render_maze(m: Maze) -> str:
    """Maze インスタンスを ASCII 文字列に変換して返す"""
    rows: list[str] = []
    # 最上段の天井
    rows.append(" " + "_" * (SIZE * 2 - 1))
    for y in range(SIZE):
        line = ["|"]  # 行の左端は必ず壁
        for x in range(SIZE):
            # 南壁（下側）
            south = m.h[y][x] if y < SIZE - 1 else True
            line.append("_" if south else " ")
            # 東壁（右側）
            east = m.v[y][x] if x < SIZE - 1 else True
            line.append("|" if east else " ")
        rows.append("".join(line))
    return "\n".join(rows)

def generate_batch() -> List[Dict]:
    if SEED is not None:
        random.seed(SEED)
    mazes = []
    for i in range(1, MAZE_COUNT + 1):
        print(f"[{i}/{MAZE_COUNT}] generating maze...", flush=True)
        m = generate_maze()
        mazes.append(maze_to_json(m, i))
        print(render_maze(m))          # ← 追加：生成した迷路を表示
        print()                        # ← 追加：行間を空けて見やすく
    return mazes

def save_batch(mazes:List[Dict]):
    os.makedirs("maze_dungeon",exist_ok=True)
    file_path=os.path.join("maze_dungeon", f"maze_{SIZE}.json")
    with open(file_path,"w",encoding="utf-8") as f:
        json.dump(mazes,f,ensure_ascii=False,indent=2)
    print(f"\n✅ Saved {len(mazes)} mazes to {file_path}")

# ─── run ───
if __name__=="__main__":
    batch=generate_batch()
    save_batch(batch)


[1/1] generating maze...
 _________________
|  _   _ _|  _    |
|   |    _|  _|_| |
| |_| |_  | |  _  |
|_     _|_   _|_  |
|_ _| | | | |   | |
|    _     _  |_  |
| |    _|  _|  _| |
| | |   |  _|_    |
|_|_|_|_|_ _|_ _|_|


✅ Saved 1 mazes to maze_dungeon\maze_9.json


In [5]:
"""
Batch Maze Generator  ── diameter display
──────────────────────────────────────────
制約
  • 全セル連結（孤立セルなし）
  • 直線通路 ≤ MAX_STRAIGHT
  • 2×2 通路禁止
  • 分岐間距離 ≤ MAX_BRANCH_DIST
  • 十字路許可・ループ率 LOOP_RATE
出力
  ./maze_dungeon/maze_<SIZE>.json
    各迷路 JSON には生成時パラメータとグラフ径も同梱
"""

# ─── adjustable parameters ───
SIZE             = 10       # 盤面の幅 = 高さ
MAX_STRAIGHT     = 10       # 連続直線通路の最大長
MAX_BRANCH_DIST  = 8        # 分岐（次数≠2）の間隔制限
LOOP_RATE        = 0.4      # ループ（迂回路）付与確率
MAZE_COUNT       = 1       # 生成する迷路数
SEED             = None     # 例: 123 で再現性確保

# ─────────────────────────────
import random, os, json
from dataclasses import dataclass
from typing import List, Tuple, Dict
from collections import deque

Cell = Tuple[int, int]                  # (x, y)
Adj  = Dict[Cell, List[Cell]]           # 隣接リスト型

@dataclass
class Maze:
    """縦・横方向の壁配列を保持"""
    v: List[List[bool]]     # 縦壁 (y, x)  True = 壁あり
    h: List[List[bool]]     # 横壁 (y, x)  True = 壁あり

# ─── 基本ユーティリティ ─────────────────────────
def neigh(x: int, y: int):
    """隣接セルを (方向, nx, ny) で列挙"""
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def carve(m: Maze, x: int, y: int, d: str):
    """指定方向の壁を取り払う（通路を掘る）"""
    if d == 'L':   m.v[y][x-1] = False
    elif d == 'R': m.v[y][x]   = False
    elif d == 'U': m.h[y-1][x] = False
    else:          m.h[y][x]   = False          # 'D'

def wall(m: Maze, x: int, y: int, d: str) -> bool:
    """その方向に壁があるか"""
    return m.v[y][x-1] if d == 'L' else \
           m.v[y][x]   if d == 'R' else \
           m.h[y-1][x] if d == 'U' else \
           m.h[y][x]

# 2×2 の完全開通を生むか判定
def makes_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    blocks = []
    if d in ('L', 'R'):
        dx = -1 if d == 'L' else 0
        for dy in (0, -1):
            blocks.append((x+dx, y+dy))
    else:                                 # U / D
        dy = -1 if d == 'U' else 0
        for dx in (0, -1):
            blocks.append((x+dx, y+dy))

    for cx, cy in blocks:
        if 0 <= cx < SIZE-1 and 0 <= cy < SIZE-1:
            v0 = not m.v[cy][cx];      v1 = not m.v[cy+1][cx]
            h0 = not m.h[cy][cx];      h1 = not m.h[cy][cx+1]
            # 仮掘削の反映
            if d == 'L' and cx == x-1 and cy in (y-1, y):   v0 = True
            if d == 'R' and cx == x   and cy in (y-1, y):   v1 = True
            if d == 'U' and cy == y-1 and cx in (x-1, x):   h0 = True
            if d == 'D' and cy == y   and cx in (x-1, x):   h1 = True
            if v0 and v1 and h0 and h1:
                return True
    return False

# 仮に掘った場合の最長直線長を算定
def max_run_after_carve(m: Maze, x: int, y: int, d: str) -> int:
    def run_h(px: int, py: int):
        run = 1; ix = px-1
        while ix >= 0 and not m.v[py][ix]: run += 1; ix -= 1
        ix = px
        while ix < SIZE-1 and not m.v[py][ix]: run += 1; ix += 1
        return run

    def run_v(px: int, py: int):
        run = 1; iy = py-1
        while iy >= 0 and not m.h[iy][px]: run += 1; iy -= 1
        iy = py
        while iy < SIZE-1 and not m.h[iy][px]: run += 1; iy += 1
        return run

    cells = [(x, y)]
    if d == 'R':   cells.append((x+1, y))
    elif d == 'L': cells.append((x-1, y))
    elif d == 'D': cells.append((x, y+1))
    else:          cells.append((x, y-1))

    return max(max(run_h(cx, cy), run_v(cx, cy)) for cx, cy in cells)

# ─── ASCII 描画 ────────────────────────────────
def render_maze(m: Maze) -> str:
    """Maze を ASCII 文字列で可視化"""
    rows: List[str] = []
    # 最上段
    rows.append(" " + "_" * (SIZE * 2 - 1))
    for y in range(SIZE):
        line = ["|"]                      # 左端は必ず壁
        for x in range(SIZE):
            south = m.h[y][x] if y < SIZE-1 else True
            east  = m.v[y][x] if x < SIZE-1 else True
            line.append("_" if south else " ")
            line.append("|" if east  else " ")
        rows.append("".join(line))
    return "\n".join(rows)

# ─── 迷路生成ロジック ──────────────────────────
def generate_tree() -> Maze:
    """深さ優先で木構造を生成（直列／分岐距離を制御）"""
    v = [[True]*(SIZE-1) for _ in range(SIZE)]
    h = [[True]*SIZE     for _ in range(SIZE-1)]
    visited = [[False]*SIZE for _ in range(SIZE)]
    degree  = [[0]*SIZE for _ in range(SIZE)]

    sx, sy = random.randrange(SIZE), random.randrange(SIZE)
    stack = [(sx, sy, '', 0, 0)]          # (x, y, prev_dir, run_len, dist_from_branch)
    visited[sy][sx] = True

    while stack:
        x, y, pd, run, dist = stack[-1]
        cand = [(d, nx, ny) for d, nx, ny in neigh(x, y) if not visited[ny][nx]]

        # 制約フィルタ
        if run >= MAX_STRAIGHT:
            cand = [t for t in cand if t[0] != pd]
        if dist >= MAX_BRANCH_DIST and pd:
            cand = [t for t in cand if len(cand) >= 2]

        cand = [t for t in cand
                if not makes_open_square(Maze(v, h), x, y, t[0])
                and max_run_after_carve(Maze(v, h), x, y, t[0]) <= MAX_STRAIGHT]

        if cand:
            d, nx, ny = random.choice(cand)
            carve(Maze(v, h), x, y, d)        # シミュレーション用 Maze を渡す
            degree[y][x]  += 1
            degree[ny][nx] += 1
            visited[ny][nx] = True
            nxt_dist = 0 if degree[ny][nx] >= 3 else dist + 1
            nxt_run  = run + 1 if d == pd else 1
            stack.append((nx, ny, d, nxt_run, nxt_dist))
        else:
            stack.pop()

    return Maze(v, h)

def add_loops(m: Maze):
    """木にループを追加して適度な巡回路を作る"""
    walls = [(x, y, 'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]] + \
            [(x, y, 'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(walls)

    for x, y, d in walls:
        if random.random() > LOOP_RATE:
            continue
        if makes_open_square(m, x, y, d):
            continue
        if max_run_after_carve(m, x, y, d) > MAX_STRAIGHT:
            continue
        carve(m, x, y, d)

def is_connected(m: Maze) -> bool:
    """全セルが連結しているか（BFS）"""
    q = [(0, 0)]
    seen = set(q)
    while q:
        x, y = q.pop()
        for d, nx, ny in neigh(x, y):
            if (nx, ny) in seen or wall(m, x, y, d):
                continue
            seen.add((nx, ny))
            q.append((nx, ny))
    return len(seen) == SIZE * SIZE

def violates(m: Maze) -> bool:
    """直線長・2×2・分岐距離の後検証"""
    # 直線チェック
    for y in range(SIZE):
        run = 1
        for x in range(SIZE-1):
            run = run + 1 if not m.v[y][x] else 1
            if run > MAX_STRAIGHT:
                return True
    for x in range(SIZE):
        run = 1
        for y in range(SIZE-1):
            run = run + 1 if not m.h[y][x] else 1
            if run > MAX_STRAIGHT:
                return True
    # 2×2 チェック
    for y in range(SIZE-1):
        for x in range(SIZE-1):
            if (not m.v[y][x] and not m.v[y+1][x] and
                not m.h[y][x] and not m.h[y][x+1]):
                return True
    # 分岐間距離チェック（次数 2 の鎖）
    deg = [[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]   += 1
                deg[y][x+1] += 1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]   += 1
                deg[y+1][x] += 1
    visited: set[Cell] = set()
    for y in range(SIZE):
        for x in range(SIZE):
            if (x, y) in visited:
                continue
            if deg[y][x] == 2:
                chain = [(x, y)]
                visited.add((x, y))
                ends = [(x, y)]
                while ends:
                    cx, cy = ends.pop()
                    for d, nx, ny in neigh(cx, cy):
                        if (nx, ny) in visited or wall(m, cx, cy, d):
                            continue
                        if deg[ny][nx] == 2:
                            chain.append((nx, ny))
                            visited.add((nx, ny))
                            ends.append((nx, ny))
                if len(chain) > MAX_BRANCH_DIST:
                    return True
            else:
                visited.add((x, y))
    return False

def generate_maze() -> Maze:
    """制約を満たす迷路が出るまで試行"""
    while True:
        m = generate_tree()
        add_loops(m)
        if is_connected(m) and not violates(m):
            return m

# ─── グラフ化 & ダイヤメーター算出 ─────────────────
def maze_to_graph(m: Maze) -> Adj:
    """Maze → 隣接リスト（無向グラフ）"""
    g: Adj = {(x, y): [] for y in range(SIZE) for x in range(SIZE)}
    for y in range(SIZE):
        for x in range(SIZE):
            for d, nx, ny in neigh(x, y):
                if not wall(m, x, y, d):
                    g[(x, y)].append((nx, ny))
    return g

def bfs_dist(g: Adj, src: Cell) -> Dict[Cell, int]:
    """src からの最短距離を BFS で取得"""
    dist = {src: 0}
    q = deque([src])
    while q:
        v = q.popleft()
        for u in g[v]:
            if u not in dist:
                dist[u] = dist[v] + 1
                q.append(u)
    return dist

def graph_diameter(g: Adj) -> Tuple[int, Cell, Cell]:
    """
    グラフ径（全頂点ペアの最短経路中で最長の長さ）を求める。
    戻り値: (長さ, 端点A, 端点B)
    """
    v0 = next(iter(g))           # 例えば (0,0)
    d0 = bfs_dist(g, v0)
    A = max(d0, key=d0.get)      # 最遠点 A

    d1 = bfs_dist(g, A)
    B = max(d1, key=d1.get)      # A から最遠点 B

    return d1[B], A, B

# ─── JSON 変換 & 保存 ───────────────────────────
def maze_to_json(m: Maze, idx: int, dia_info: Tuple[int, Cell, Cell]) -> Dict:
    dia_len, A, B = dia_info
    v_walls = [[x, y] for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]
    h_walls = [[x, y] for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    return {
        "id": f"maze{idx:03}",
        "size": SIZE,
        "max_straight": MAX_STRAIGHT,
        "max_branch_dist": MAX_BRANCH_DIST,
        "loop_rate": LOOP_RATE,
        "diameter": dia_len,
        "dia_endpoints": [A, B],
        "v_walls": v_walls,
        "h_walls": h_walls
    }

def generate_batch() -> List[Dict]:
    """迷路を生成しつつ ASCII・ダイヤメーターを表示"""
    if SEED is not None:
        random.seed(SEED)
    mazes_json: List[Dict] = []

    for i in range(1, MAZE_COUNT + 1):
        print(f"[{i}/{MAZE_COUNT}] generating maze...", flush=True)
        m = generate_maze()
        g = maze_to_graph(m)
        dia_len, A, B = graph_diameter(g)

        print(render_maze(m))
        print(f"    ↳ グラフ径（最長最短路長）: {dia_len}  ( {A} → {B} )\n")

        mazes_json.append(maze_to_json(m, i, (dia_len, A, B)))
    return mazes_json

def save_batch(mazes: List[Dict]):
    os.makedirs("maze_dungeon", exist_ok=True)
    file_path = os.path.join("maze_dungeon", f"maze_{SIZE}.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(mazes, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved {len(mazes)} mazes to {file_path}")

# ─── main ───────────────────────────────────────
if __name__ == "__main__":
    batch = generate_batch()
    save_batch(batch)


[1/1] generating maze...
 ___________________
|  _    |  _|   |_  |
|_   _| |    _| |  _|
| |_  |   | |   |  _|
|_ _ _ _|_|_  | | | |
| | |  _|   |_|    _|
|  _ _ _ _|   | |_  |
| |  _  |_  | |     |
|_   _    |  _  | | |
|    _  |    _     _|
|_|_ _ _ _|_ _ _|_ _|
    ↳ グラフ径（最長最短路長）: 31  ( (0, 4) → (0, 1) )

✅ Saved 1 mazes to maze_dungeon\maze_10.json
